# Review misclassified examples

Load packages

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
from sklearn.metrics import confusion_matrix

Load Model

In [ ]:
model = tf.keras.models.load_model('/tmp/ml-pipeline/model/digit-recognition.keras')

Load data

In [ ]:
import boto3
from boto3 import session
import os

key_id = os.environ.get("AWS_ACCESS_KEY_ID")
secret_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
bucket_name = os.environ.get("AWS_S3_BUCKET")
s3_endpoint = os.environ.get("AWS_S3_ENDPOINT")

s3_client = boto3.client("s3", aws_access_key_id=key_id, aws_secret_access_key=secret_key, endpoint_url=s3_endpoint)

In [ ]:
X_val = np.load('/tmp/ml-pipeline/data/X_val.npy')
y_val = np.load('/tmp/ml-pipeline/data/y_val.npy')
X_train = np.load('/tmp/ml-pipeline/data/X_train.npy')

Predict

In [ ]:
y_val_pred = np.argmax(model.predict(X_val), axis=1)
y_val_pred

In [ ]:
y_val_true = np.argmax(y_val,axis=1)
y_val_true

Confusion matrix

In [ ]:
cm = confusion_matrix(np.argmax(y_val,axis=1), y_val_pred)

In [ ]:
cm_index = cm
np.fill_diagonal(cm_index,0)
cm_index = np.where(cm_index==cm_index.max())
if len(cm_index)>1:
    cm_index = [cm_index[0][0],cm_index[1][0]]
cm_index

In [ ]:
cm_index

In [ ]:
nine_incorrect_examples = X_val[(y_val_true==cm_index[0]) & (y_val_pred==cm_index[1])]
nine_incorrect_examples = nine_incorrect_examples.reshape(-1,28,28)

In [ ]:
filters_layer_1 = model.layers[1].get_weights()[0]
filters_layer_1 = np.squeeze(filters_layer_1).reshape(32,5,5)

fig = plt.figure(figsize=(10, 10))
fig.suptitle('Missclassified examples')
w = 10
h = 10
columns = 6
rows = 6
for i in range(columns*rows +1):
    if i==nine_incorrect_examples.shape[0]:
        break
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(nine_incorrect_examples[i], cmap='Greys')
    plt.title('True:{},Pred:{}'.format(cm_index[0],cm_index[1]))
    plt.xticks([])
    plt.yticks([])
plt.show()

> <div class="alert alert-block alert-info">
<b>Looking at the missclassifications, its understandable why the model was unable classify these examples correctly. Some of the examples are quite ambiguous, even a human labeller would probably be unable to clearly label them with good confidence. However, it does seem that there is room for improvement for some of the examples. With the use of data augmentation and hyper-parameter tuning, we should be able to further improve performance.</b>
</div>

> <div class="alert alert-block alert-info">
<b>Before we try these additional techniques, let's take a quick look at the learnt convolutional filters and feature maps, which should give us some insight on how the network is learning.</b></div>

In [ ]:
model.layers

In [ ]:
model.layers[1].get_weights()[0].shape

In [ ]:
filters_layer_1 = model.layers[1].get_weights()[0]
filters_layer_1 = np.squeeze(filters_layer_1).reshape(32,5,5)

fig = plt.figure(figsize=(10, 10))
fig.suptitle('CNN 1 FILTERS')
w = 10
h = 10
columns = 6
rows = 6
for i in range(columns*rows +1):
    if i==filters_layer_1.shape[0]:
        break
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(filters_layer_1[i], cmap='Greys')
    plt.title(i)
    plt.xticks([])
    plt.yticks([])
plt.show()

> <div class="alert alert-block alert-info">
<b>Shown above is the visualizatins of the 32 learnt filters in the first convolutional layer. The filters are what convolve over the images to produce the activation maps, also known as feature-maps.</b>
    <br></br>
<b>The feature-maps capture key features and patterns from the input (i.e the input image or feature maps from previous ConvBlocks).</b>
    <br></br> 
<b>The reason for visualizing feature maps is to understand what features of the input are detected and preserved in the feature maps. Lets now explore some of the feature maps to see what the model is capturing.</b>
</div>

In [ ]:
for i in range(len(model.layers)):
    layer = model.layers[i]
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # summarize output shape
    print(i, layer.name, layer.output.shape)

In [ ]:


successive_outputs = [layer.output for layer in model.layers[1:]]
fm_model = keras.Model(inputs=model.input, outputs=successive_outputs)

successive_outputs


In [ ]:
test_example = X_train[[9]]
plt.imshow(test_example[0], cmap='Greys')
plt.colorbar()
plt.show()
successive_feature_maps = fm_model.predict(test_example)
len(successive_feature_maps) # 12 for 12 layers

In [ ]:
layer_names = [layer.name for layer in model.layers]
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    print(feature_map.shape)
    print(layer_name)
    if len(feature_map.shape) == 4:
        n_features = feature_map.shape[-1]  # number of features in the feature map
        size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)

        # We will tile our images in this matrix
        display_grid = np.zeros((size, size * n_features))

        # Postprocess the feature to be visually palatable
        for i in range(n_features):
            x  = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std ()
            x *=  64
            x += 128
            x  = np.clip(x, 0, 255).astype('uint8')
            # Tile each filter into a horizontal grid
            display_grid[:, i * size : (i + 1) * size] = x
            
        # Display the grid
        scale = 20. / n_features
        plt.figure( figsize=(scale * n_features, scale) )
        plt.title(layer_name) 
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='Greys' )
        plt.colorbar()
        
plt.show()